In [57]:
import torch
import json
import transformer as t
import importlib
importlib.reload(t);

In [58]:
#
# Hyper parameter
#
topic           = 'commits'
token_size      = 3
batch_size      = 128
max_iterations  = 4000
eval_interval   = 2000
learning_rate   = 1e-3
eval_iters      = 200

#
# Network 
#
t.attention_heads_per_block = 16
t.attention_blocks          = 16
t.sample_size               = 48     # number of consecutive characters to predict from
t.embedding_size            = 256    # size of the embedding vectors
t.dropout                   = 0.2

# remember to set t.vocabulary_size when data is loaded

In [59]:
# read in file
with open(f'{topic}/{topic}.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text = text.ljust((len(text) + token_size-1)//token_size*token_size, '\n')

tokens = list(set([text[i:i+token_size] for i in range(0, len(text), token_size)]))
t.vocabulary_size = len(tokens)

# Build encoder and decoder
token_to_index = {token:index for index,token in enumerate(tokens)}
index_to_token = {index:token for index,token in enumerate(tokens)}
def encode(s):
    s = s.ljust((len(s) + token_size-1)//token_size * token_size, '\n')
    return [token_to_index[token] for token in [s[i:i+token_size] for i in range(0, len(s), token_size)]]
encode = lambda s: [token_to_index[token] for token in [s[i:i+token_size] for i in range(0, len(s), token_size)]]
decode = lambda l: ''.join([index_to_token[i] for i in l])

# save vocabulary
dictionary = {str(k): v for k, v in index_to_token.items()}
with open(f'{topic}/vocabulary.json', 'w') as file:
    json.dump(dictionary, file)

# create data tensors
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
training_data = data[:n]
validation_data = data[n:]

In [60]:
#
# Create model
#
torch.manual_seed(1337)
model = t.Transformer()
m = model.to(t.device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
start_iteration = 0
sum(p.nelement() for p in model.parameters())

20754897

In [ ]:

start_iteration = 11000

# Load the states
checkpoint = torch.load(f'{topic}/{topic}-{start_iteration}.nn');
model.load_state_dict(checkpoint['model_state_dict']);
optimizer.load_state_dict(checkpoint['optimizer_state_dict']);

model.train()

In [61]:
#
# mini-batch creation
#
def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - t.sample_size, (batch_size, ))
    x = torch.stack([data[i:i+t.sample_size] for i in ix])
    y = torch.stack([data[i+1:i+t.sample_size+1] for i in ix])
    x,y = x.to(t.device), y.to(t.device)
    return x,y
#
# evaluation
#
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def save_model(step):
    model.eval()
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'{topic}/{topic}-{step}.nn');
    model.train()

def export_model(step):
    model.eval()
    dummy_input = torch.randint(low=0, high=t.vocabulary_size, size=(1, t.sample_size), dtype=torch.long)
    torch.onnx.export(model, dummy_input, f"{topic}/{topic}-{step}.onnx");
    model.train()    

In [62]:
#
# Training
#
model.train()
for iter in range(max_iterations):
    current_iteration = iter + start_iteration
    if current_iteration % eval_interval == 0 and (start_iteration == 0 or current_iteration > start_iteration):
        print(f"{current_iteration}: evaluating...")
        losses = estimate_loss()
        print(f"{current_iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")    

    if current_iteration % 1000 == 0 and (start_iteration == 0 or current_iteration > start_iteration):
        print(f"{current_iteration}: saving checkpoint...")
        save_model(current_iteration);
        export_model(current_iteration);
        print(f"{current_iteration}: checkpoint saved.")

    if current_iteration % 250 == 0:
        print(f"{current_iteration}: training")

    xb, yb = get_batch('train')
    _, loss = model(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

current_iteration += 1
print(f"{current_iteration}: evaluating...")
losses = estimate_loss()
print(f"{current_iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
print(f"{current_iteration}: saving checkpoint...")
save_model(current_iteration)
export_model(current_iteration)
print(f"{current_iteration}: checkpoint saved.")

0: evaluating...
0: train loss 9.8347, val loss 9.8321
0: saving checkpoint...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

0: checkpoint saved.
0: training


### Commit message generation training

#### Hyper and network parameters
| Parameter                 | Value 
| :--------                 | ----:
| token size                | 3
| sample size               | 48
| embedding size            | 256
| batch size                | 128
| learning rate             | 1e-3
| attention heads per block | 16
| attention blocks          | 16
| dropout ratio             | 0.2


#### Training results
| Iteration | Loss (training)   | Loss (validation)
| :-------: | :-------------:   | :---------------:
| 0         | 9.8347            | 9.8321

In [ ]:
model.eval()
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=t.device), max_tokens=500)[0].tolist()))